In [301]:
import pandas as pd
import math
import os
from datetime import datetime

In [302]:
# Constants
initial_rating = 1500
initial_rd = 350
tau = 0.5
q = math.log(10) / 400
c = 1.5

# Glicko Functions
def g(rd):
    return 1 / (1 + 3 * (q**2) * (rd**2) / (math.pi**2))**0.5

def E(r, ri, rdi):
    return 1 / (1 + 10**(g(rdi) * (ri - r) / 400))

def d2(r, ri, rdi):
    inner = (g(rdi)**2) * E(r, ri, rdi) * (1 - E(r, ri, rdi))
    return 1 / ((q**2) * inner)

def new_rd(rd, d2):
    return 1 / ((1 / rd**2) + (1 / d2))**0.5

def new_rating(r, rd, ri, rdi, s, d2):
    return r + (q / ((1 / rd**2) + (1 / d2))) * g(rdi) * (s - E(r, ri, rdi))

# Function to increase RD over time due to inactivity
def increase_rd_over_time(player_ratings, current_time):
    for player, (rating, rd, last_active) in player_ratings.items():
        # Calculate time elapsed in days
        days_inactive = (current_time - last_active).days
        if days_inactive > 0:
            # Increase RD based on time elapsed
            new_rd = min(math.sqrt(rd**2 + c**2 * days_inactive), initial_rd)
            player_ratings[player] = (rating, new_rd, last_active)

def update_ratings(player_ratings, match_results, current_time):
    error = 0
    increase_rd_over_time(player_ratings, current_time)
    for match in match_results:
        player1, player2, result = match
        r1, rd1, last_active1 = player_ratings.get(player1, (initial_rating, initial_rd, current_time))
        r2, rd2, last_active2 = player_ratings.get(player2, (initial_rating, initial_rd, current_time))
        if result == 1:
            error += (r1 < r2) * E(r1, r2, rd2)
        else:
            error += (r1 > r2) * E(r2, r1, rd1)

        d2_1 = d2(r1, r2, rd2)
        d2_2 = d2(r2, r1, rd1)
        
        new_rd1 = new_rd(rd1, d2_1)
        new_rd2 = new_rd(rd2, d2_2)
        
        new_r1 = new_rating(r1, rd1, r2, rd2, result, d2_1)
        new_r2 = new_rating(r2, rd2, r1, rd1, 1 - result, d2_2)
        
        player_ratings[player1] = (new_r1, new_rd1, current_time)
        player_ratings[player2] = (new_r2, new_rd2, current_time)
    return error

players = {}
matches = [("player1", "player2", 1), ("player2", "player1", 0)]
update_ratings(players, matches, datetime.now())
print(players)


{'player1': (1720.1602564648724, 260.27316726259585, datetime.datetime(2024, 6, 29, 9, 2, 24, 945987)), 'player2': (1279.8397435351276, 260.27316726259585, datetime.datetime(2024, 6, 29, 9, 2, 24, 945987))}


In [303]:
tf = pd.read_csv('data/tournaments_wtt.tsv', sep='\t', parse_dates=['StartDateTime']).sort_values(['StartDateTime'])
tf

,EventName,EventType,EventTypeId,Country,City,ContinentCode,Subcontinent,StartDateTime,EndDateTime,EventId
3,WTT Contender Doha 2021,WTT Contender,34,Qatar,Doha,asia,west asia,2021-02-28 11:30:00,2021-03-06T23:59:59,2410
4,WTT Star Contender Doha 2021,WTT Star Contender,35,Qatar,Doha,asia,west asia,2021-03-05 19:30:00,2021-03-13T23:59:00,2411
1,Tokyo 2020 Olympic Games,Olympic Games,38,Japan,Tokyo,asia,east asia,2021-07-23 00:00:00,2021-08-08T00:00:00,2345
10,WTT Contender Budapest 2021,WTT Contender,34,Hungary,Budapest,europe,eastern europe,2021-08-15 00:00:00,2021-08-20T00:00:00,2487
7,2021 ITTF Czech International Open,ITTF International Open,80,Czechia,Olomouc,europe,eastern europe,2021-08-21 00:00:00,2021-08-25T00:00:00,2480
...,...,...,...,...,...,...,...,...,...,...
135,WTT Feeder Caracas 2024,WTT Feeder,81,Venezuela,Caracas,americas,south america,2024-10-30 00:00:00,2024-11-03T00:00:00,2912
115,WTT Champions Frankfurt 2024,WTT Champions,65,Germany,Frankfurt,europe,western europe,2024-11-03 00:00:00,2024-11-10T00:00:00,2877
127,WTT Feeder Düsseldorf II 2024,WTT Feeder,81,Germany,Düsseldorf,europe,western europe,2024-11-18 00:00:00,2024-11-22T00:00:00,2890
145,WTT Finals Fukuoka 2024,WTT Finals,75,Japan,Kitakyushu,asia,east asia,2024-11-20 00:00:00,2024-11-24T00:00:00,2947


In [304]:
directory = os.fsencode('data/wtt_cleaned/matches')

players = {}

error = 0
for event in tf.itertuples():
    if not os.path.isfile(f'data/wtt_cleaned/matches/{event.EventId}.tsv'):
        continue

    df = pd.read_csv(f'data/wtt_cleaned/matches/{event.EventId}.tsv', sep='\t', parse_dates=['start'])
    # event_id	doc	fmt	gender	stage	stage_id	duration	start	a_id	b_id	x_id	y_id	res_a	res_x
    # 2345	TTEMTEAM--------------FNL-00010001--------	T	M	FNL	00010001	1560	2021-08-06 10:34:29	110267	105649	102832	101222	3	0
    matches = []
    for row in df[df.fmt != 'D'].itertuples():
        matches.append((row.a_id, row.x_id, int(row.res_a > row.res_x)))        
    error += update_ratings(players, matches, event.StartDateTime)

resf = pd.DataFrame.from_dict(players, orient='index')
resf

,0,1,2
107028,1747.283338,56.116518,2024-06-19
121582,1800.660512,61.161373,2024-05-30
107445,1502.125948,56.624867,2024-06-11
109002,1310.121324,63.962741,2024-06-11
117821,1848.217765,66.952657,2024-06-11
...,...,...,...
209663,1358.149016,251.885656,2024-06-19
113568,1359.348666,263.434387,2024-06-19
202262,1282.998274,253.718019,2024-06-19
123121,1245.052825,283.877237,2024-06-19


In [305]:
error

1591.3226489505948

In [306]:
pf = pd.read_csv('data/wtt_cleaned/players.tsv', sep='\t').set_index('id')
pf

,org,name,gender
id,,,
100001,IND,ANTHONY Amalraj,M
100032,EGY,ABDEL-AZIZ Farah,W
100079,NGR,ABIODUN Bode,M
100089,RSA,ABRAHAMS Luke,M
100154,TOG,AGBETOGLO Mawussi,M
...,...,...,...
209660,NGR,RABIU Aishat,W
209663,GHA,YEBOAH Michael,M
209805,CHN,FENG Yixuan,X


In [307]:
rf = resf.join(pf)
srf = rf.sort_values(0, ascending=False)

In [308]:
import json

with open('./ittf_ranking/06252024_m.json', 'r') as f:
    data = json.load(f)
    mrank = {
        p: i + 1
        for i, p in enumerate(data)
    }

with open('./ittf_ranking/06252024_w.json', 'r') as f:
    data = json.load(f)
    wrank = {
        p: i + 1
        for i, p in enumerate(data)
    }

In [309]:
w100 = srf[(srf.gender == 'W') & (srf[1] <= 100)].head(100)
w100[0] = w100[0].round().astype('int')
w100[1] = w100[1].round().astype('int')
w100.reset_index(inplace=True)
w100.drop(columns=['index', 'gender', 2], inplace=True)
w100.reset_index(inplace=True)
w100.rename(columns={'index': 'rank', 0: 'rating', 1: 'conf'}, inplace=True)
w100['rank'] += 1
w100['wttrank'] = pd.Series(wrank.get(name) or wrank.get(' '.join(reversed(name.split(' ', 1))), 9999) for name in w100.name).astype('int')
w100['wttdiff'] = w100.wttrank - w100['rank']
w100.to_csv('top100F.tsv', sep='\t', index=False)

In [310]:
with open('top100f.txt', 'w') as f:
    f.write('|rank|rating|±dev|org|name|wtt|diff|\n')
    f.write('|:-|:-|:-|:-|:-|:-|:-|\n')
    for row in w100.set_index('rank').itertuples():
        f.write('|'+ '|'.join((str(r) for r in row)) + '|\n')

In [311]:
w100 = srf[
    (srf.gender == 'M')
    & (srf[1] <= 100)
].head(100)
w100[0] = w100[0].round().astype('int')
w100[1] = w100[1].round().astype('int')
w100.reset_index(inplace=True)
w100.drop(columns=['index', 'gender', 2], inplace=True)
w100.reset_index(inplace=True)
w100.rename(columns={'index': 'rank', 0: 'rating', 1: 'conf'}, inplace=True)
w100['rank'] += 1
w100['wttrank'] = pd.Series(mrank.get(name) or mrank.get(' '.join(reversed(name.split(' ', 1))), 9999) for name in w100.name).astype('int')
w100['wttdiff'] = w100.wttrank - w100['rank']
w100.to_csv('top100M.tsv', sep='\t', index=False)

In [312]:
with open('top100m.txt', 'w') as f:
    f.write('|rank|rating|±dev|org|name|wtt|diff|\n')
    f.write('|:-|:-|:-|:-|:-|:-|:-|\n')
    for row in w100.set_index('rank').itertuples():
        f.write('|'+ '|'.join((str(r) for r in row)) + '|\n')

In [313]:
rf[rf.name == 'OJIO Haruna']

,0,1,2,org,name,gender
135179,1841.997079,145.740748,2023-08-22,JPN,OJIO Haruna,W


In [314]:
# manual fixes
# Jia Nan YUAN -> YUAN Jia Nan
# SUH Hyo Won -> SUH Hyowon
# YANG Ha Eun -> YANG Haeun
# LEE Sang Su -> LEE Sangsu

In [315]:
directory = os.fsencode('data/wtt_cleaned/matches')

players = {}

events = []
for event in tf.itertuples():
    if not os.path.isfile(f'data/wtt_cleaned/matches/{event.EventId}.tsv'):
        continue

    events.append(
        pd.read_csv(f'data/wtt_cleaned/matches/{event.EventId}.tsv', sep='\t', parse_dates=['start'])
    )

matchf = pd.concat(events)
matchf

,event_id,doc,fmt,gender,stage,stage_id,duration,start,a_id,b_id,x_id,y_id,res_a,res_x
0,2410,TTEMSINGLES-----------FNL-000100--,S,M,FNL,100,2884,2021-03-06 11:40:00,107028,NaN,121582,NaN,4,1
1,2410,TTEMSINGLES-----------RND2001100----------,S,M,RND2,1100,1225,2021-03-01 08:20:00,107445,NaN,109002,NaN,3,0
2,2410,TTEWDOUBLES-----------FNL-000100--,D,W,FNL,100,1481,2021-03-06 07:00:00,117820,110752.0,114706,114105.0,3,0
3,2410,TTEWSINGLES-----------FNL-000100--,S,W,FNL,100,3464,2021-03-06 11:00:00,117821,NaN,123672,NaN,4,2
4,2410,TTEXDOUBLES-----------FNL-000100--,D,X,FNL,100,3201,2021-03-06 08:20:00,121582,110797.0,105136,137894.0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2868,TTEXDOUBLES-----------8FNL000700----------,D,X,8FNL,700,1184,2024-06-20 09:35:00,123143,113565.0,209954,202971.0,0,3
105,2868,TTEXDOUBLES-----------8FNL000500----------,D,X,8FNL,500,1458,2024-06-20 09:00:00,209955,203618.0,131412,132715.0,0,3
106,2868,TTEXDOUBLES-----------8FNL000100----------,D,X,8FNL,100,1141,2024-06-20 09:00:00,117357,123977.0,202982,203471.0,3,0
107,2868,TTEXDOUBLES-----------8FNL000200----------,D,X,8FNL,200,1055,2024-06-20 09:00:00,113563,102445.0,118702,123129.0,3,1


In [316]:
xyb = matchf[(matchf.fmt != 'D') & ((matchf.a_id == 117821) | (matchf.x_id == 117821))]

In [317]:
wnames = xyb.merge(pf[['name']].rename(columns={'name': 'name_a'}), left_on='a_id', right_index=True).merge(pf[['name']].rename(columns={'name': 'name_x'}), left_on='x_id', right_index=True)

In [318]:
wnames.to_csv('tmp.tsv', sep='\t', index=False)